<a href="https://colab.research.google.com/github/novis10813/Portfolio-Optimizer/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

跑完這邊直接拿
1. encoded_train{}
2. encoded_test{}

兩個都是dict 型態，內部皆儲存3維Tensor
可用encoded_train['Asset name'] 和 
encoded_test['Asset name']直接訪問


In [233]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader


window_length = 60
test_samples = int(814*0.2)
asset_list = ['SPY', 'IVV', 'VTI', 'VOO', 'QQQ', 'BTC', 'ETH', 'USDC-USD']

In [234]:
"""
Read data and make percentage change
"""
prices_data = pd.read_csv('/content/close_price.csv')

df = prices_data.copy().drop(['Unnamed: 0'], axis = 1)
df = df.set_index(['date'])

df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.tail(len(df_pct_change) - 1)

In [235]:
"""
Get the total array of data
"""



scaler = MinMaxScaler()
total_data_list = {}
for asset in asset_list:
        
    total= np.array([scaler.fit_transform(df_pct_change[asset].values[i-window_length:i].reshape(-1, 1)) for i in tqdm(range(window_length,len(df_pct_change)))])
    new_name = '{}'.format(asset)
    total_data_list[new_name] = total

100%|██████████| 754/754 [00:00<00:00, 3114.24it/s]


In [280]:
"""
Train Test Split
"""


close_train = {}
close_test = {}


for keys in tqdm(list(total_data_list.keys())):
        x_test = total_data_list[keys][-test_samples:]
        x_train = total_data_list[keys][:-test_samples]
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')

        new_name_train = '{}'.format(keys)        
        new_name_test = '{}'.format(keys)
        close_train[new_name_train] = x_train
        close_test[new_name_test] = x_test   

100%|██████████| 8/8 [00:00<00:00, 3607.23it/s]


In [282]:
"""
Transfer data into tensor
"""



dataloader_train = {}
dataloader_test = {}

for keys in tqdm(list(close_train.keys())):
        train = DataLoader(close_train[keys], batch_size=window_length, num_workers=0)
        dataloader_train_new_name = '{}'.format(keys)
        dataloader_train[dataloader_train_new_name] = train

for keys in tqdm(list(close_test.keys())):
        test = DataLoader(close_test[keys], batch_size=window_length, num_workers=0)
        dataloader_test_new_name = '{}'.format(keys)
        dataloader_test[dataloader_test_new_name] = test


#dataloader = DataLoader(close_train['SPY'], batch_size=window_length, num_workers=0)
#for data in dataloader:
  #print(data.size())


100%|██████████| 8/8 [00:00<00:00, 7273.89it/s]


In [364]:
"""
Convolutional Autoencoder
"""
from torch.nn.modules.activation import Sigmoid
from torchsummary import summary


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16,kernel_size=3,padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool1d(2, padding=0),  # b, 16, 5, 5
            nn.Conv1d(in_channels = 16, out_channels = 1, kernel_size = 3, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool1d(2, padding = 0)  # b, 8, 2, 2
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=1, out_channels = 1, kernel_size = 3, padding=1),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.Upsample(30),
            nn.ConvTranspose1d(in_channels =1, out_channels = 16, kernel_size = 1),  # b, 1, 28, 28
            nn.ReLU(True),
            nn.Upsample(60),
            nn.ConvTranspose1d(in_channels =16, out_channels = 1, kernel_size = 3,padding=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
model = autoencoder()
summary(model, (1,60))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 60]              64
              ReLU-2               [-1, 16, 60]               0
         MaxPool1d-3               [-1, 16, 30]               0
            Conv1d-4                [-1, 1, 30]              49
              ReLU-5                [-1, 1, 30]               0
         MaxPool1d-6                [-1, 1, 15]               0
   ConvTranspose1d-7                [-1, 1, 15]               4
              ReLU-8                [-1, 1, 15]               0
          Upsample-9                [-1, 1, 30]               0
  ConvTranspose1d-10               [-1, 16, 30]              32
             ReLU-11               [-1, 16, 30]               0
         Upsample-12               [-1, 16, 60]               0
  ConvTranspose1d-13                [-1, 1, 60]              49
          Sigmoid-14                [-1

In [365]:
"""
parameters
"""
num_epochs = 100
learning_rate = 1e-3
criterion = nn.BCELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)

In [366]:
"""
Model training
"""

train_loss = 0
for epoch in range(num_epochs):
  for keys in tqdm(list(close_train.keys())):
    for data in dataloader_train[keys]:       
       # ===================forward=====================
        
        data = torch.transpose(data,1,2)
        optimizer.zero_grad()
        outputs = model(data)
        # perform a single optimization step (parameter update)
        loss = criterion(outputs, data)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()
       
    # ===================log========================
    train_loss = train_loss/len(dataloader_train['SPY'])
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
          epoch, 
          train_loss
          ))
    

 50%|█████     | 4/8 [00:00<00:00, 16.25it/s]

Epoch: 0 	Training Loss: 0.688332
Epoch: 0 	Training Loss: 0.753734
Epoch: 0 	Training Loss: 0.758082
Epoch: 0 	Training Loss: 0.758201


 75%|███████▌  | 6/8 [00:00<00:00, 15.75it/s]

Epoch: 0 	Training Loss: 0.764608
Epoch: 0 	Training Loss: 0.786706
Epoch: 0 	Training Loss: 0.781893


100%|██████████| 8/8 [00:00<00:00, 14.59it/s]


...

Epoch: 49 	Training Loss: 0.729754
Epoch: 49 	Training Loss: 0.727052
Epoch: 49 	Training Loss: 0.726312


 75%|███████▌  | 6/8 [00:00<00:00, 14.06it/s]

Epoch: 49 	Training Loss: 0.726931
Epoch: 49 	Training Loss: 0.735971
Epoch: 49 	Training Loss: 0.722623


100%|██████████| 8/8 [00:00<00:00, 13.96it/s]

Epoch: 49 	Training Loss: 0.721997
Epoch: 49 	Training Loss: 0.753195


In [391]:
"""
Get encoded Training Data
"""

encoded_train ={}
for keys in tqdm(list(total_data_list.keys())):
    input = torch.tensor(close_train[keys])
    input = torch.transpose(input, 1, 2)
    output = model.encoder(input)
    encoded_train_new_name = '{}'.format(keys)
    encoded_train[encoded_train_new_name] = output


100%|██████████| 8/8 [00:00<00:00, 110.31it/s]


In [395]:
"""
Get the Encoded Testing data
"""


encoded_test ={}
for keys in tqdm(list(total_data_list.keys())):
    input = torch.tensor(close_test[keys])
    input = torch.transpose(input, 1, 2)
    output = model.encoder(input)
    encoded_test_new_name = '{}'.format(keys)
    encoded_test[encoded_test_new_name] = output

100%|██████████| 8/8 [00:00<00:00, 223.21it/s]
